In [4]:
import numpy as np

def get_fingerprint_45(structure):
    elements = [site.specie for site in structure.sites]
    unique_elements = list(set(elements))

    feature_dict = {
        "atomic_number": [],
        "atomic_mass": [],
        "period": [],
        "group": [],
        "row": [],
        "column": [],
        "covalent_radius": [],
        "vdw_radius": [],
        "electronegativity": [],
        "atomic_volume": [],
        "melting_point": [],
        "boiling_point": [],
        "density": [],
        "electron_affinity": [],
        "fusion_enthalpy": [],
        "vaporization_enthalpy": [],
        "thermal_conductivity": [],
        "specific_heat_capacity": [],
        "electrical_conductivity": [],
        "heat_of_formation": [],
        "hardness": [],
        "youngs_modulus": [],
        "bulk_modulus": [],
        "shear_modulus": [],
        "poisson_ratio": [],
        "brinell_hardness": [],
        "mohs_hardness": [],
        "vickers_hardness": [],
        "density_of_states_at_fermi_level": [],
        "band_gap": [],
        "polarizability": [],
        "magnetic_moment": [],
        "refractive_index": [],
        "oxidation_state": [],
        "ionic_radius": [],
        "magnetic_ordering": [],
        "crystal_structure": [],
        "space_group_number": [],
        "space_group_symbol": [],
        "crystal_system": [],
        "number_of_atoms": [],
        "number_of_elements": [],
        "packing_fraction": [],
    }

    # 模拟填 0
    # 你之前代码里是根据 element 周期表提特征
    for key in feature_dict.keys():
        feature_dict[key] = [0.0]

    fingerprint = np.array([np.mean(vals) for vals in feature_dict.values()])
    return fingerprint

def get_fingerprint_120(structure):
    # 假设 stoich120 是 stoich45 的增强版（你之前代码也是）
    fp45 = get_fingerprint_45(structure)
    # 这里我们举例：假装加上一些额外特征（补零即可）
    extra_features = np.zeros(120 - len(fp45))
    fingerprint = np.concatenate([fp45, extra_features])
    return fingerprint


ModuleNotFoundError: No module named 'qmof_fingerprint'